In [44]:
from datasets import load_dataset,concatenate_datasets

In [93]:
feature_column = ["tokens","ner_tags"]
split_list = ["train","validation","test"]

def remove_columns_from_dataset_dict(dataset_dict,feature_columns):
    assert sorted(split_list) == sorted(list(dataset_dict.keys())), "Dataset is not containing all splits for train,test,val"
    for split in split_list:
        remove_column_list = [col for col in list(dataset_dict[split].features) if col not in feature_column ]
        dataset_dict[split] = dataset_dict[split].remove_columns(remove_column_list)
    return dataset_dict


def merging_all_splits_from_dataset_dict(dataset1,dataset2):
    for split in split_list:
        assert dataset1[split].features.type == dataset2[split].features.type
        dataset1[split] = concatenate_datasets([dataset1[split],dataset2[split]])
    return dataset1    
            

# Preprocessing `wikiann`

In [ ]:
wikiann= load_dataset("wikiann","en")

In [80]:
additional_selected_validation_wikiann = wikiann["validation"].train_test_split(test_size=0.5)
additional_selected_test_wikiann = wikiann["test"].train_test_split(test_size=0.5)

In [81]:
assert wikiann["train"].features.type == additional_selected_validation_wikiann["train"].features.type

In [82]:
wikiann["train"] = concatenate_datasets([additional_selected_test_wikiann["train"],wikiann["train"]])
wikiann["validation"] = additional_selected_validation_wikiann["test"]
wikiann["test"] = additional_selected_test_wikiann["test"]

In [85]:
wikiann_cleaned = remove_columns_from_dataset_dict(wikiann,feature_column)

In [86]:
wikiann_cleaned

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 25000
    })
})

# Preprocessing `conll2003`

In [87]:
conll = load_dataset("conll2003")

Reusing dataset conll2003 (/Users/philipp/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


In [88]:
conll_cleaned = remove_columns_from_dataset_dict(conll,feature_column)

In [89]:
conll_cleaned

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

# Merging the datasets

In [94]:
merged_dataset = merging_all_splits_from_dataset_dict(conll_cleaned,wikiann_cleaned)

ValueError: External features info don't match the dataset:
Got
{'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'pos_tags': Sequence(feature=ClassLabel(num_classes=47, names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], names_file=None, id=None), length=-1, id=None), 'chunk_tags': Sequence(feature=ClassLabel(num_classes=23, names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], names_file=None, id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER